In [ ]:
import sys
import time
import datetime
import os
import tensorflow as tf
import argparse
import json
import socket
from PIL import Image
from util import *
import numpy as np
%load_ext autoreload
%autoreload 2

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
with open("hyperParams.json") as f:
    instanceParams = json.load(f)

In [ ]:
instanceParams

In [ ]:
logPath = "logs/"
snapshotPath = "snapshots/"

In [ ]:
printFrequency = instanceParams["printFreq"]
snapshotFrequency = instanceParams["snapFreq"]
batchSize = instanceParams["batchSize"]

iterations = instanceParams["iterations"]
baseLearningRate = instanceParams["baseLR"]
learningRate = baseLearningRate
snapshotFrequency = instanceParams["snapshotFreq"]

In [ ]:
iterations = 1000

In [ ]:
resume, startIteration, snapshotFiles = checkResume(snapshotPath,logPath, {'resume': 'n'})

Below is the code used to generate train image txt files from DAVIS's train.txt.

In [ ]:
# img_list = None
# with open('../example_data/data/DAVIS/ImageSets/480p/train.txt', 'r') as f:
#     img_list = f.readlines()
# img_list = [x.strip() for x in img_list]
# img_list = [item.split(' ')[0] for item in img_list]
# img_list = ["data/DAVIS" + item for item in img_list]
# img_list

# objects = sorted(['bear','car-shadow','elephant','lucia','rollerblade','blackswan','car-turn','flamingo','mallard-fly','scooter-black',
# 'bmx-bumps','cows','goat','mallard-water','scooter-gray','bmx-trees','dance-jump','hike','motocross-bumps','soapbox','boat','dance-twirl',
# 'hockey','motocross-jump','soccerball','breakdance','dog','horsejump-high','motorbike','stroller','breakdance-flare',
# 'dog-agility','horsejump-low','paragliding','surf','bus','drift-chicane','kite-surf','paragliding-launch','swing','camel',
# 'drift-straight','kite-walk','parkour','tennis','car-roundabout','drift-turn','libby','rhino','train'])

# objects = ['/' + item + '/' for item in objects]
# count = {}
# for item in objects:
#     count[item] = 0

# for im in img_list:
#     for item in objects:
#         if item in im:
#             count[item] += 1

# for key in objects:
#     try:
#         if count[key] == 0:
#             del count[key]
#     except:
#         print key


# i = 0
# with open('../example_data/datalists/train_im0.txt', 'w') as f:
#     for key in sorted(count.keys()):
#         l = count[key]
#         for j in range(l-1):
#             f.write("%s\n" % img_list[i])
#             i += 1
#         i += 1

# i = 0
# with open('../example_data/datalists/train_im1.txt', 'w') as f:
#     for key in sorted(count.keys()):
#         l = count[key]
#         i += 1
#         for j in range(l-1):
#             f.write("%s\n" % img_list[i])
#             i += 1

In [ ]:
with tf.device("/gpu:0"):
    trainingData = TrainingData(batchSize,instanceParams)

In [ ]:
with tf.device("/gpu:0"):
    # init
    with tf.variable_scope("netShare"):
        networkBodyF = NetworkBody(trainingData,instanceParams)
    with tf.variable_scope("netShare",reuse=True):
        networkBodyB = NetworkBody(trainingData,instanceParams,flipInput=True)

    trainingLoss = TrainingLoss(instanceParams,networkBodyF,networkBodyB,trainingData)
    solver,learningRateTensor = attachSolver(trainingLoss.loss)

    # loss scheduling
    recLossBWeightTensor = trainingLoss.recLossBWeight

In [ ]:
# merge summaries
merged = tf.summary.merge_all()

# saver
saver = tf.train.Saver(max_to_keep=0)

In [ ]:
from dotmap import DotMap
arg = DotMap()
arg.logDev = False

In [ ]:
printFrequency = 20

### Training

In [ ]:
# start
with sessionSetup(arg) as sess:
#     if resume:
#         saver.restore(sess,snapshotPath+snapshotFiles[-1][:-6])
#     else:
#         sess.run(tf.initialize_all_variables())
    saver.restore(sess,
                  '../model_download_scripts/photometric_smoothness/weights/iter_0000000000500000.ckpt')

    trainingData.dataQueuer.start_queueing(sess)

    #start summary writer
    summary_writer = tf.summary.FileWriter(logPath, sess.graph)

    #run
    lastPrint = time.time()
    for i in range(500000, 500000 + iterations):
        # scheduled values
        learningRate = learningRateSchedule(baseLearningRate, i)
        recLossBWeight = unsupLossBSchedule(i)

         #run training
        feed_dict = {
            learningRateTensor: learningRate,
            recLossBWeightTensor: recLossBWeight,
        }
        summary,result,totalLoss = sess.run([merged,solver,trainingLoss.loss], feed_dict=feed_dict)

        if (i+1) % printFrequency == 0:
            timeDiff = time.time() - lastPrint
            itPerSec = printFrequency/timeDiff
            remainingIt = iterations - i
            eta = remainingIt/itPerSec
            print("Iteration "+str(i+1)+": loss: "+str(totalLoss)+", iterations per second: "+str(itPerSec)+", ETA: "+str(datetime.timedelta(seconds=eta)))+", lr: "+str(learningRate)

            summary_writer.add_summary(summary,i+1)
            summary_writer.flush()
            lastPrint = time.time()

        if (i+1) % snapshotFrequency == 0:
            saver.save(sess,"snapshots/iter_"+str(i+1).zfill(16)+".ckpt")

        sys.stdout.flush()

    #close queing
    trainingData.dataQueuer.close(sess)

### Visualization

This is just a hack to view a random estimated flow ...

In [ ]:
# start
with sessionSetup(arg) as sess:
    saver.restore(sess,
                  '../model_download_scripts/photometric_smoothness/weights/iter_0000000000500000.ckpt')

    trainingData.dataQueuer.start_queueing(sess)

    #start summary writer
    summary_writer = tf.summary.FileWriter(logPath, sess.graph)

    #run
    flowFinal = networkBodyF.flows[0]
    flowViz = flowToRgb(flowFinal)
    for i in range(500000, 500000 + 1):
        # scheduled values
        learningRate = learningRateSchedule(baseLearningRate, i)
        recLossBWeight = unsupLossBSchedule(i)

         #run training
        feed_dict = {
            learningRateTensor: learningRate,
            recLossBWeightTensor: recLossBWeight,
        }
        flow,summary,result,totalLoss = sess.run([flowViz,merged,solver,trainingLoss.loss], feed_dict=feed_dict)

    # close queing
    trainingData.dataQueuer.close(sess)

In [ ]:
arr = np.minimum(np.asarray(flow),1)
arr = np.maximum(arr,0)
arr = np.squeeze(np.asarray(arr*255,np.uint8))
im = Image.fromarray(arr[0])
im